In [1]:
# Import from meg_qc, relative to the path of this file

# add meg_qc module to the path

from meg_qc.meg_qc_pipeline import make_derivative_meg_qc

config_file_path = 'meg_qc/settings.ini' 
internal_config_file_path='meg_qc/settings_internal.ini' # internal settings in in
raw, raw_cropped_filtered_resampled, QC_derivs, QC_simple, df_head_pos, head_pos, scores_muscle_all1, scores_muscle_all2, scores_muscle_all3, raw1, raw2, raw3, avg_ecg, avg_eog = make_derivative_meg_qc(config_file_path, internal_config_file_path)


Using matplotlib as 2D backend.
___MEG QC___:  list_of_subs ['Ab140232', 'Al150424', 'Cb140229', 'Cc160310', 'Jl150443', 'Lb160367', 'Mb160304', 'Mk150295', 'Mm150194', 'Mp110340', 'Mp150285', 'Rt160359', 'Sl160372', 'emptyroom']
___MEG QC___:  TOTAL subs 14
___MEG QC___:  EMPTY room? Ab140232 emptyroom
___MEG QC___:  Dataset:  /Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922
___MEG QC___:  Take SID:  Al150424
___MEG QC___:  list_of_fifs ['/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922/sub-Al150424/ses-01/meg/sub-Al150424_ses-01_acq-crosstalk_meg.fif', '/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922/sub-Al150424/ses-01/meg/sub-Al150424_ses-01_task-mcd_run-02_meg.fif', '/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922/sub-Al150424/ses-01/meg/sub-Al150424_ses-01_task-mcd_run-03_meg.fif', '/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922/sub-Al150424/ses-01/meg/sub-Al150424_ses-01_task-mcd_run-04_meg.fif', '/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922

/Volumes/MAIN DATA/Users/evgeniiagapontseva/Local/MEG-QC-code/meg_qc/source/initial_meg_qc.py:636: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = mne.io.read_raw_fif(data_file, allow_maxshield=True, on_split_missing='ignore')


<Raw | sub-Al150424_ses-01_task-mcd_run-03_meg.fif, 342 x 311000 (311.0 s), ~5.4 MB, data not loaded>

Reading 0 ... 30000  =      0.000 ...    30.000 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1.4e+02 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 140.00 Hz: -6.02 dB

___MEG QC___:  Data filtered from 0.0 to 140.0 Hz.
Trigger channel has a non-zero initial value of 43 (consider using initial_event=True to detect this event)
43 events found
Event IDs: [ 1  2 41 43 45 47]
Trigger channel has a non-zero initial value of 43 (consider using initial_event=True to detect this event)
43 events found
Event IDs: [ 1  2 41 43 45 47]
___MEG QC___:  Data resampled to 1000 Hz. 
___MEG QC___:  Stimulus channels detected: ['STI001', 'STI002', 'STI003', 'STI004', 'STI005', 'STI006', 'STI007', 'STI008', 'STI009', 'STI010', 'STI011', 'STI012', 'STI013', 'STI014', 'STI015', 'STI016', 'STI101', 'STI201', 'STI301']
NOTE: pick

In [ ]:
import mne
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003922/sub-Al150424/ses-01/meg/sub-Al150424_ses-01_task-mcd_run-03_meg.fif', allow_maxshield=True)
stim=['STI001', 'STI002', 'STI003', 'STI004', 'STI005', 'STI006', 'STI007', 'STI008', 'STI009', 'STI010', 'STI011', 'STI012', 'STI013', 'STI014', 'STI015', 'STI016', 'STI101', 'STI201', 'STI301']

mne.find_events(raw, min_duration=0.2, stim_channel=stim)

In [ ]:

def find_t0_channels(artif_per_ch):

    #run peak detection on all channels and find the 5 channels with the highest peaks:
    
    first_crests = []
    first_crests_magnitudes = []

    for artif_per_ch.artif_data in artif_per_ch:
        peaks_pos_loc = find_peaks(artif_per_ch.artif_data)
        peaks_neg_loc = find_peaks(-artif_per_ch.artif_data)
        peaks_pos_magn = artif_per_ch.artif_data[peaks_pos_loc]
        peaks_neg_magn = artif_per_ch.artif_data[peaks_neg_loc]
        #find the highest of all peak_pos:
        max_peak_pos_loc = peaks_pos_loc[np.argmax(peaks_pos_magn)]
        min_peak_neg_loc = peaks_neg_loc[np.argmin(peaks_neg_magn)]

        #put these 2 together and sort by which comes first:
        crests = np.sort(np.concatenate((max_peak_pos_loc, min_peak_neg_loc)))

        first_crests.append(crests[0])
        first_crests_magnitudes.append(artif_per_ch.artif_data[crests[0]])

    #find the 5 channels with the highest first_crests_magnitudes:
    first_crests_magnitudes = np.array(first_crests_magnitudes)
    first_crests = np.array(first_crests)
    first_crests_sorted = first_crests[np.argsort(first_crests_magnitudes)]
    first_crests_sorted = first_crests_sorted[-5:]

    #find the average location of the first crest over the 5 channels:
    t0_channels = int(np.mean(first_crests_sorted))

    return t0_channels

def find_t0_mean(mean_rwave):
    
    #run peak detection on the mean_rwave and find the first peak:
    peaks_pos_loc = find_peaks(mean_rwave)
    peaks_neg_loc = find_peaks(-mean_rwave)
    peaks_pos_magn = mean_rwave[peaks_pos_loc]
    peaks_neg_magn = mean_rwave[peaks_neg_loc]
    #find the highest of all peak_pos:
    max_peak_pos_loc = peaks_pos_loc[np.argmax(peaks_pos_magn)]
    min_peak_neg_loc = peaks_neg_loc[np.argmin(peaks_neg_magn)]

    #put these 2 together and sort by which comes first:
    crests = np.sort(np.concatenate((max_peak_pos_loc, min_peak_neg_loc)))

    t0_mean = crests[0]

    return t0_mean


def find_t0_shift(t0_channels, t0_mean):
    t0_shift = t0_mean - t0_channels
    return t0_shift


def shift_mean_wave(mean_rwave, t0_shift):
    mean_rwave_shifted = np.roll(mean_rwave, t0_shift)
    return mean_rwave_shifted

#Now plot the mean_rwave_shifted and the mean_rwave on the same plot to see if they are aligned
# Plot each of the channels on another plot

def plot_mean_rwave_shifted(mean_rwave_shifted, mean_rwave):
    #plot using plotly

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(len(mean_rwave_shifted)), y=mean_rwave_shifted, mode='lines', name='mean_rwave_shifted'))
    fig.add_trace(go.Scatter(x=np.arange(len(mean_rwave)), y=mean_rwave, mode='lines', name='mean_rwave'))
    fig.show()

def plot_channels(artif_per_ch):
    #plot using plotly

    fig = go.Figure()
    for artif_per_ch.artif_data in artif_per_ch:
        fig.add_trace(go.Scatter(x=np.arange(len(artif_per_ch.artif_data)), y=artif_per_ch.artif_data, mode='lines', name='artif_per_ch.artif_data'))
    fig.show()

def align_mean_rwave(mean_rwave, artif_per_ch):

    #Get 5 highest channels.
    #get highest positive peak and highest negative peak
    # put them in list sorted by which peak comes first

    #calculate average location of the 1st peak over 5 channels and set it as t0

    #do the same for the mean wave. set it s first peak as t0

    #calculate time shift between the two t0s

    #shift the mean wave by the time shift

    t0_channels = find_t0_channels(artif_per_ch)
    t0_mean = find_t0_mean(mean_rwave)
    t0_shift = find_t0_shift(t0_channels, t0_mean)
    mean_rwave_shifted = shift_mean_wave(mean_rwave, t0_shift)
    plot_mean_rwave_shifted(mean_rwave_shifted, mean_rwave)
    plot_channels(artif_per_ch)

    return mean_rwave_shifted
    






In [ ]:
#Aligned Wave Shapes:

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Generate two aligned wave shapes
time = np.linspace(0, 1, 100)
wave1 = np.sin(2 * np.pi * 2 * time)
wave2 = np.sin(2 * np.pi * 2 * time)

# Calculate correlation
correlation = np.correlate(wave1, wave2, mode='same')
corr1 = pearsonr(wave1, wave2)
print(corr1)

# Plot the wave shapes and correlation
plt.subplot(2, 1, 1)
plt.plot(time, wave1)
plt.title('Wave 1')
plt.subplot(2, 1, 2)
plt.plot(time, wave2)
plt.title('Wave 2')
plt.show()


#Misaligned Wave Shapes:

import numpy as np
import matplotlib.pyplot as plt

# Generate two misaligned wave shapes
time = np.linspace(0, 1, 100)
wave1 = np.sin(2 * np.pi * 2 * time)
wave2 = np.sin(2 * np.pi * 2 * (time + 0.15))  # Shifted by 0.2 seconds

# Calculate correlation
correlation = np.correlate(wave1, wave2, mode='same')
corr2 = pearsonr(wave1, wave2)
print(corr2)

# Plot the wave shapes and correlation
plt.subplot(2, 1, 1)
plt.plot(time, wave1)
plt.title('Wave 1')
plt.subplot(2, 1, 2)
plt.plot(time, wave2)
plt.title('Wave 2')
plt.show()

In [ ]:
#Create wave shapes
import plotly.graph_objects as go
import numpy as np

# Define the number of points in each array
num_points = 100

# Create an array of time values
t = np.linspace(0, 2*np.pi, num_points)

# Define the amplitudes for the R-wave shapes
amplitudes = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5]

# Define the maximum time shift in seconds
max_shift = 0.4

# Create five arrays with R-wave shapes, shifted in time
waves = []
for i, amplitude in enumerate(amplitudes):
    # Generate a random time shift within the maximum shift range
    time_shift = np.random.uniform(-max_shift, max_shift)
    
    # Shift the time values
    shifted_t = t + time_shift
    
    # Create the R-wave shape with the shifted time values
    wave = np.exp(-shifted_t) * np.sin(4*shifted_t) * amplitude
    #waves.append(wave)
    waves.append(wave[::-1])


fig = go.Figure()
for i, wave in enumerate(waves):
    fig.add_trace(go.Scatter(x=time, y=wave, name=f'Wave {i+1}'))
fig.show()

In [ ]:
import numpy as np

# Assuming you have two arrays: array1 and array2
array1 = waves[0]
array2 = waves[5]

import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

# Assuming you have two arrays: array1 and array2

# Find peaks in both arrays
peaks1, _ = find_peaks(array1)
peaks2, _ = find_peaks(array2)

# Calculate the time shift based on the peak positions
time_shift = peaks1[0] - peaks2[0]

# Shift array2 to align with array1
aligned_array2 = np.roll(array2, time_shift)

# Create the figure
fig = go.Figure()

# Add the array1 trace
fig.add_trace(go.Scatter(x=np.arange(len(array1)), y=array1, name='Array 1'))

# Add the array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(array2)), y=array2, name='Array 2'))

# Add the aligned_array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(aligned_array2)), y=aligned_array2, name='Aligned Array 2'))

# Set the layout
fig.update_layout(title='Aligned Arrays using Peak Detection',
                  xaxis_title='Time',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

array1 = waves[0]
array2 = -waves[5]

# Create the figure
fig = go.Figure()

# Add the array1 trace
fig.add_trace(go.Scatter(x=np.arange(len(array1)), y=array1, name='Array 1'))

# Add the array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(array2)), y=array2, name='Array 2'))

# Set the layout
fig.update_layout(title='Aligned Arrays using Peak Detection',
                  xaxis_title='Time',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()


In [ ]:


# Assuming you have two arrays: array1 and array2

# Find peaks in array1
peaks1, _ = find_peaks(array1)

# Initialize variables for best alignment
best_time_shift = 0
best_correlation = -np.inf
best_aligned_array2 = None

# Try aligning array2 in both orientations
for flip in [False, True]:
    # Flip array2 if needed
    #aligned_array2 = np.flip(array2) if flip else array2
    aligned_array2 = -array2 if flip else array2

    # Find peaks in aligned_array2
    peaks2, _ = find_peaks(aligned_array2)

    # Calculate the time shift based on the peak positions
    time_shift = peaks1[0] - peaks2[0]

    # Shift aligned_array2 to align with array1
    aligned_array2 = np.roll(aligned_array2, time_shift)

    # Calculate the correlation between array1 and aligned_array2
    correlation = np.corrcoef(array1, aligned_array2)[0, 1]

    # Update the best alignment if the correlation is higher
    if correlation > best_correlation:
        best_correlation = correlation
        best_time_shift = time_shift
        best_aligned_array2 = aligned_array2

# Create the figure
fig = go.Figure()

# Add the array1 trace
fig.add_trace(go.Scatter(x=np.arange(len(array1)), y=array1, name='Array 1'))

# Add the array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(array2)), y=array2, name='Array 2'))

# Add the best_aligned_array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(best_aligned_array2)), y=best_aligned_array2, name='Aligned Array 2'))

# Set the layout
fig.update_layout(title='Aligned Arrays with Flipped Second Array',
                  xaxis_title='Time',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()

In [ ]:
import numpy as np
from scipy.signal import find_peaks

avg_ecg_epoch_data_nonflipped_limited_to_event = np.array(waves)

max_values=np.max(np.abs(avg_ecg_epoch_data_nonflipped_limited_to_event), axis=1)
print(max_values)
max_values_ind=np.argsort(max_values)[::-1] 
print(max_values_ind)
max_values_ind=max_values_ind[:5]

chosen_5 = (avg_ecg_epoch_data_nonflipped_limited_to_event[max_values_ind])

thresh_lvl_peakfinder = 5


#get the highest peak for every channel:
max_amplitude1 = []
index_of_max_amplitude1=[]
for ch_data in avg_ecg_epoch_data_nonflipped_limited_to_event:

    thresh_mean=(max(ch_data) - min(ch_data)) / thresh_lvl_peakfinder
    peak_locs_pos, _ = find_peaks(ch_data, prominence=thresh_mean)
    peak_locs_neg, _ = find_peaks(-ch_data, prominence=thresh_mean)

    all_peaks = np.concatenate((peak_locs_pos, peak_locs_neg))
    print('all peaks', all_peaks)

    #Find the peak with the maximal amplitude:

    max_amplitude_peak = np.argmax(np.abs(ch_data[all_peaks]))

    #now find the index of this point in the channel data:
    index_of_max_amplitude1.append(all_peaks[max_amplitude_peak])

    print('Index1', index_of_max_amplitude1)

    #now find the magnitude of the data in this point:

    max_amplitude1.append(ch_data[index_of_max_amplitude1[-1]])

    

# find 5 channels which have the highest peaks and get the locations of these peaks:
highest_channels_sorted = np.argsort(max_amplitude1)[::-1] 
print(highest_channels_sorted)
max_ind_of_chosen_5=highest_channels_sorted[:5]

print(max_ind_of_chosen_5)


In [ ]:
index_of_max_amplitude2=[]
for ch_data in max_ind_of_chosen_5:

    thresh_mean=(max(ch_data) - min(ch_data)) / thresh_lvl_peakfinder
    peak_locs_pos, _ = find_peaks(ch_data, prominence=thresh_mean)
    peak_locs_neg, _ = find_peaks(-ch_data, prominence=thresh_mean)

    all_peaks = np.concatenate((peak_locs_pos, peak_locs_neg))
    print('all peaks', all_peaks)

    #Find the peak with the maximal amplitude:

    max_amplitude_peak = np.argmax(np.abs(ch_data[all_peaks]))


    #6. Output the index of the point with the maximal amplitude:

    index_of_max_amplitude1.append(all_peaks[max_amplitude_peak])
    print('Index1', index_of_max_amplitude1)

    if len(all_peaks)>1:
        #7. Now find the second largest peak:
        all_peaks_without_max = np.delete(all_peaks, max_amplitude_peak)

        print('no max', all_peaks_without_max)

        max_amplitude_peak = np.argmax(np.abs(ch_data[all_peaks_without_max]))


        #6. Output the index of the point with the maximal amplitude:

        index_of_max_amplitude2.append(all_peaks_without_max[max_amplitude_peak])
        print('Index2', index_of_max_amplitude2)
        
    else:
        index_of_max_amplitude2.append(np.nan)

mean_index_of_max_amplitude1 = np.nanmean(index_of_max_amplitude1)

# If in more than a half of cases there was no second biggest peak found, skip it and assign t) as first peak:
non_zero_count = np.count_nonzero(index_of_max_amplitude2)
percentage = (non_zero_count/len(index_of_max_amplitude2)) * 100

if percentage < 50:
    t0_peak = int(mean_index_of_max_amplitude1)
else:
    mean_index_of_max_amplitude2 = np.nanmean(index_of_max_amplitude2)
    #Now out of them set the first peak (according to time) as t0.
    t0_peak = int(np.nanmin([mean_index_of_max_amplitude1, mean_index_of_max_amplitude2]))


print('mean_ind1', mean_index_of_max_amplitude1)
print('mean_ind2', mean_index_of_max_amplitude2)


print(t0_peak)



In [ ]:
import numpy as np

arr = np.array([5, 2, 9, 1, 7, 3])

# Get the indices that would sort the array in ascending order
sorted_indices = np.argsort(arr)

# Index of the largest value
largest_index = sorted_indices[-1]

# Index of the second largest value
second_largest_index = sorted_indices[-2]

print("Index of the largest value:", largest_index)
print("Index of the second largest value:", second_largest_index)

In [ ]:
import mne
from IPython.display import display

# Load the MEG data
raw=mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif', preload=True)

display(raw)


In [ ]:

# Define the EOG channel names
eog_channels = ['EOG 061', 'EOG 062']

# extract the data of 2 EOG channels
eog_data = raw.copy().pick_channels(eog_channels).get_data()

print(eog_data)


In [ ]:

# Plot the data with plotly:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a figure with two subplots
fig = make_subplots(rows=2, cols=1)

x_values = raw.times

# Add a trace for the first subplot
fig.add_trace(go.Scatter(x=x_values, y=eog_data[0], mode='lines', name='EOG 1'), row=1, col=1)

# Add a trace for the second subplot
fig.add_trace(go.Scatter(x=x_values, y=eog_data[1], mode='lines', name='EOG 2'), row=2, col=1)

# Update the layout
fig.update_layout(title='EOG Data', xaxis_title='Time (s)', yaxis_title='Amplitude')

# Show the figure
fig.show()



In [ ]:
import numpy as np

# create two arrays
array1 = np.array([1, 2, 3, 4, 5])
array2 = np.array([6, 7, 8, 9, 10])

# stack the arrays horizontally
stacked = np.stack((array1, array2), axis=0)

display(stacked)

# calculate the covariance matrix
covariance_matrix = np.cov(stacked)

print(covariance_matrix)


In [ ]:
import mne
import numpy as np
from scipy.signal import find_peaks

# Load the MEG data
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif')
#raw=mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif', preload=True)

# Select the EOG channels
eog_channels = mne.pick_types(raw.info, meg=False, eeg=False, stim=False, eog=True)

# Get the names of the EOG channels
eog_channel_names = [raw.ch_names[ch] for ch in eog_channels]

print('EOG channel names:', eog_channel_names)

eog_events = mne.preprocessing.find_eog_events(raw)

In [ ]:

picks_ECG = mne.pick_types(raw.info, ecg=True)
ecg_ch_name = [raw.info['chs'][name]['ch_name'] for name in picks_ECG]

arr=raw.get_data(picks=ecg_ch_name)[0] 
height = np.mean(arr) + 1 * np.std(arr)
fs=raw.info['sfreq']
peaks, _ = find_peaks(arr, height=height, distance=round(0.5 * fs)) #assume there are no peaks within 0.5 seconds from each other.
ecg_events = peaks/fs

# Define the time window of interest
time_window = [0.2, 0.2]  # in seconds
tmin=-0.2
tmax=0.2

# Convert time window to samples
sfreq = 1000  # sampling frequency of your data
time_window_samples = np.round(np.array(time_window) * sfreq).astype(int)
print('samples', time_window_samples)

# Initialize an empty array to store the extracted epochs
epochs = np.zeros((len(peaks), int((tmax-tmin)*sfreq)))

print('HERE')
print(arr)
print(epochs)

# Loop through each ECG event and extract the corresponding epoch
for i, event in enumerate(peaks):
    start = event - time_window_samples[0]
    start = np.round(event + tmin*sfreq).astype(int)
    end = event + time_window_samples[1]
    end= np.round(event + tmax*sfreq).astype(int)
    epochs[i, :] = arr[start:end]

#average all epochs:
avg_ecg=np.mean(epochs, axis=0)

#print average ecg with plotly:
import plotly.graph_objects as go

fig = go.Figure()
#create time vector based on time window and sampling frequency:
times= np.arange(tmin, tmax, 1/sfreq)
fig.add_trace(go.Scatter(x=times, y=avg_ecg, mode='lines', name='ECG'))
fig.show()




In [ ]:

# Detect the R-wave peaks in the filtered ECG channel data
r_peaks, ch_ecg, pulse, ecg_data_rec = mne.preprocessing.find_ecg_events(raw, return_ecg=True)
print(ecg_data_rec)

#plot the ECG data with plotly:
import plotly.graph_objects as go

times=[t for t in range(len(ecg_data_rec[0]))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=ecg_data_rec[0], mode='lines', name='ECG'))
fig.update_layout(title='ECG data', xaxis_title='Time (s)', yaxis_title='ECG (mV)')
fig.show()

# Calculate the time difference between each R-wave peak and the first R-wave peak
r_wave_epochs = (r_peaks - r_peaks[0]) / raw.info['sfreq']
print('r_wave_epochs', r_wave_epochs)

# Calculate the average R-wave epoch
avg_r_wave_epoch = np.mean(r_wave_epochs)
print('avg_r_wave_epoch', avg_r_wave_epoch)

if ecg_ch_name:
    # Extract the ECG channel data
    ecg_data, times = raw.get_data(ecg_ch_name, return_times=True)
    ecg_data2=ecg_data_rec
else:
    ecg_data=ecg_data_rec



# Use the average R-wave epoch to extract a segment of data from the ECG channel
avg_r_wave_data = ecg_data[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]
avg_r_wave_data2 = ecg_data2[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]

#plot the average R-wave epoch with plotly:
import plotly.graph_objects as go

times=[t/raw.info['sfreq'] for t in range(len(avg_r_wave_data[0]))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data[0], mode='lines', name='ECG'))
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data2[0], mode='lines', name='ECG2'))
fig.update_layout(title='Average R-wave epoch', xaxis_title='Time (s)', yaxis_title='ECG (mV)')
fig.show()

raw

In [ ]:
print([t/raw.info['sfreq'] for t in range(len(avg_r_wave_data[0]))])

In [ ]:
ecg_data
avg_r_wave_epoch * raw.info['sfreq']
r_peaks[0]

In [ ]:
avg_r_wave_data = ecg_data[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]
avg_r_wave_data2 = ecg_data2[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]

#plot the average R-wave epoch with plotly:
import plotly.graph_objects as go

times=[t/raw.info['sfreq'] for t in range(len(avg_r_wave_data[0]))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data[0], mode='lines', name='ECG'))
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data2[0], mode='lines', name='ECG2'))
fig.update_layout(title='Average R-wave epoch', xaxis_title='Time (s)', yaxis_title='ECG (mV)')
fig.show()

In [ ]:
import numpy as np
from scipy.stats import pearsonr

# Generate two waves
wave1 = np.array([1, 2, 3, 4, 5])
wave2 = np.array([2, 4, 6, 8, 10])

# Calculate the Pearson correlation coefficient and p-value
corr_coef, p_value = pearsonr(wave1, wave2)

# Print the results
print("Pearson correlation coefficient:", corr_coef)
print("p-value:", p_value)


#plot both waves with plotly:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=wave1, mode='lines', name='wave1'))
fig.add_trace(go.Scatter(y=wave2, mode='lines', name='wave2'))
fig.show()


In [ ]:
import random

lobe_colors = {
        'Left Frontal': '#1f77b4',
        'Right Frontal': '#ff7f0e',
        'Left Temporal': '#2ca02c',
        'Right Temporal': '#9467bd',
        'Left Parietal': '#e377c2',
        'Right Parietal': '#d62728',
        'Left Occipital': '#bcbd22',
        'Right Occipital': '#17becf'}

print(random.choice(list(lobe_colors.values())))

In [ ]:
from scipy.ndimage import gaussian_filter
import numpy as np

# Generate some noisy wave data
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x) + np.random.normal(0, 0.5, 100)

# Apply Gaussian smoothing with a sigma of 2
y_smooth = gaussian_filter(y, sigma=4)

# Plot the original and smoothed waves
import matplotlib.pyplot as plt
plt.plot(x, y, label='Noisy wave')
plt.plot(x, y_smooth, label='Smoothed wave')
plt.legend()
plt.show()


In [ ]:
import plotly.graph_objs as go
import pandas as pd

# create sample data
df = pd.DataFrame({'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, index=['A']*10)
df = df.T
print(df)

# create box plot trace
box_trace = go.Box(x=df.iloc[0], orientation='h')
#box_trace = go.Box(x=df['values'], y=df.index, orientation='h', name='')

fig = go.Figure(data=box_trace)

for col in df.columns:
    fig.add_trace(go.Scatter(x=df[col], name=col))

# for v in df['values']:
#     #fig.add_trace(go.Scatter(x=df['values'], y=df.index, mode='markers', marker=dict(size=5, color='yellow'), name='Scatter Plot', hovertext=df.index))
#     fig.add_trace(go.Scatter(x=[v], y=['A'], mode='markers', marker=dict(size=5, color='yellow'), name='Scatter Plot', hovertext=df.index))

# plot figure
fig.show()


In [ ]:
import plotly.graph_objs as go

fig = go.Figure()

# Add box plot trace
fig.add_trace(go.Box(x=[1, 2, 3, 4, 5]))

# Add horizontal line at y=0
fig.update_layout(
    shapes=[
        dict(
            type='line',
            yref='y',
            y0=0,
            y1=0,
            xref='paper',
            x0=0,
            x1=1
        )
    ]
)

fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Create example dataset
np.random.seed(123)
std_val = pd.DataFrame({'Group': ['A', 'A', 'B', 'B', 'C', 'C'],
                     'Value': np.random.normal(size=6)})

# Create box plot with custom marker colors
fig = go.Figure()
fig.add_trace(go.Box(x=std_val['Group'], y=std_val['Value'], name='Value',
                     marker=dict(color='red')))

# Update layout
fig.update_layout(title='Box plot with custom marker colors',
                  xaxis_title='Group', yaxis_title='Value')

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objs as go

# create a box plot with custom marker color
trace = go.Box(
    y=[1, 2, 3, 4, 5],
    marker=dict(
        color='blue'
    )
)

# create a figure and add the box plot trace
fig = go.Figure(data=[trace])

# show the figure
fig.show()


In [ ]:
#This here is to save all the average ECG/EOG data into a pickle file, so I can test difefrent wave detection algorythms on them without running the pipeline again

import pickle 
import plotly.graph_objects as go
import numpy as np


# open a file in write binary mode
with open("avg_ecg.pkl", "wb") as f:
    # dump the list of objects into the file
    pickle.dump(avg_ecg, f)

with open("avg_eog.pkl", "wb") as f:
    # dump the list of objects into the file
    pickle.dump(avg_eog, f)




In [ ]:
#This here is to open the pickle files from above and plot the data

import pickle 
import plotly.graph_objects as go
import numpy as np

# open the file in read binary mode
with open("avg_ecg0.pkl", "rb") as f:
    # load the list of objects from the file
    eog_list = pickle.load(f)

print(eog_list[0])

sfreq=1000
t = np.round(np.arange(-0.4, 0.4+1/sfreq, 1/sfreq), 3) #yes, you need to round
fig0=go.Figure()
for x in range(0, len(eog_list)):
    fig_temp=eog_list[x].plot_epoch_and_peak(t, 'Channels affected by ECG artifact: ', 'mag', fig0)
    for trace in fig_temp['data']:
        fig0.add_trace(trace)

fig0.update_layout(
    yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e')) 
fig0.show()


In [ ]:
# same for EOG

import pickle 
import plotly.graph_objects as go
import numpy as np
from copy import deepcopy
from scipy.ndimage import gaussian_filter


# open the file in read binary mode
with open("avg_eog0.pkl", "rb") as f:
    # load the list of objects from the file
    eog_list = pickle.load(f)

sfreq=1000
t = np.round(np.arange(-0.4, 0.4+1/sfreq, 1/sfreq), 3) #yes, you need to round
fig0=go.Figure()
for x in range(0, len(eog_list)):
    fig_temp=eog_list[x].plot_epoch_and_peak(t, 'Channels affected by ECG artifact: ', 'mag')
    for trace in fig_temp['data']:
        fig0.add_trace(trace)

fig0.update_layout(
    yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e')) 
fig0.show()

#Now apply the gaussia filter to each trace and plot result in the same figure:
fig0_new=deepcopy(fig0)
for trace in fig0_new['data']:
    y=trace['y']
    y_smooth = gaussian_filter(y, sigma=10)
    trace['y']=y_smooth

fig0_new.show()

In [ ]:
import mne
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

In [ ]:
#show sensor posiions using mne:

import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

mne.viz.plot_sensors(raw.info, kind='topomap', ch_type='grad', show_names=True, ch_groups='position')


In [ ]:
#PLOT SENSORS IN 2D with plotly
import numpy as np
import plotly.graph_objects as go
import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)


mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels_objs = {'mag': mag_ch_names, 'grad': grad_ch_names}

# Get the sensor locations
sensor_locs = raw.info['chs']
#print(sensor_locs)
#coords_mag=[loc['loc'][:2] for loc in sensor_locs]
coords_mag=[loc['loc'] for loc in sensor_locs if loc['ch_name'] in mag_ch_names]
#print(len(coords), coords)
print(len(coords_mag), coords_mag)

x = [r[0] for r in coords_mag]
y = [r[1] for r in coords_mag]
#x, y, z = [loc['loc'][:3] for loc in sensor_locs]
names = [loc['ch_name'] for loc in sensor_locs if loc['ch_name'] in mag_ch_names]
kinds= [loc['kind'] for loc in sensor_locs]
print(kinds)

# Create a scatter plot of the sensor locations
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', text=names))

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000)

# Set the plot title and axis labels
fig.update_layout(title='MEG Sensor Locations', xaxis_title='X', yaxis_title='Y')

# Add a circle shape to the plot to show the position of the head
fig.update_layout(
    shapes=[
        dict(
            type='circle',
            xref='x',
            yref='y',
            x0=-0.1,
            y0=-0.1,
            x1=0.1,
            y1=0.12,
            line=dict(color='red', width=2),
            opacity=0.5
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[0, -0.02],
            y0=[0.1, 0.08],
            line=dict(color='black', width=2)
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[0, 0.02],
            y0=[0.1, 0.08],
            line=dict(color='black', width=2)
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[-0.02, 0.02],
            y0=[0.08, 0.08],
            line=dict(color='black', width=2))
])



# Show the plot 
fig.show()

In [ ]:
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

#PLOT 3 D

def switch_names_on_off(fig):
    # Define the buttons
    buttons = [
    dict(label='Show channel names when hovering',
         method='update',
         args=[{'mode': 'markers'}]),
    dict(label='Always show channel names',
         method='update',
         args=[{'mode': 'markers+text'}])
    ]

    # Add the buttons to the layout
    fig.update_layout(updatemenus=[dict(type='buttons',
                                        showactive=True,
                                        buttons=buttons)])

    return fig



# Extract the sensor locations and names for magnetometers
mag_locs = raw.copy().pick_types(meg='mag').info['chs']
mag_pos = [ch['loc'][:3] for ch in mag_locs]
mag_names = [ch['ch_name'] for ch in mag_locs]

# Create the magnetometer plot with markers only

mag_fig = go.Figure(data=[go.Scatter3d(x=[pos[0] for pos in mag_pos],
                                       y=[pos[1] for pos in mag_pos],
                                       z=[pos[2] for pos in mag_pos],
                                       mode='markers',
                                       marker=dict(size=5),
                                       text=mag_names,
                                       hovertemplate='%{text}')],
                                       layout=go.Layout(width=1000, height=1000))

mag_fig.update_layout(title='Magnetometers')

mag_fig = switch_names_on_off(mag_fig)
mag_fig.show()



# Extract the sensor locations and names for gradiometers
grad_locs = raw.copy().pick_types(meg='grad').info['chs']
grad_pos = [ch['loc'][:3] for ch in grad_locs]
grad_names = [ch['ch_name'] for ch in grad_locs]

#since grads have 2 sensors located in the same spot - need to put their names together to make pretty plot labels:

grad_pos_together = []
grad_names_together = []

for i in range(len(grad_pos)-1):
    if all(x == y for x, y in zip(grad_pos[i], grad_pos[i+1])):
        grad_pos_together += [grad_pos[i]]
        grad_names_together += [grad_names[i]+', '+grad_names[i+1]]
    else:
        pass


# Add both sets of gradiometer positions to the plot:
grad_fig = go.Figure(data=[go.Scatter3d(x=[pos[0] for pos in grad_pos_together],
                                        y=[pos[1] for pos in grad_pos_together],
                                        z=[pos[2] for pos in grad_pos_together],
                                        mode='markers',
                                        marker=dict(size=5),
                                        text=grad_names_together,
                                        hovertemplate='%{text}')],
                                        layout=go.Layout(width=1000, height=1000))

grad_fig.update_layout(title='Gradiometers')


# Add the button to have names show up on hover or always:
grad_fig = switch_names_on_off(grad_fig)

# Show the plots

grad_fig.show()


In [ ]:

# MUSCLE ARIFACTS IN EMPTYROOM DATA:
# Discussed with Andreas:
# We can see very high muscle scores at the very beginning and end of the empty room recording
# Are these real muscle artifacts or filtering errors?
# Cut out 1st second of the data where they are visible.
# make Fourier transform of the 1st second and see if there are high amplitudes visible for the muscle frequencies - nope 
# most likely this is filtering.
# next, follow the MNE steps for muscle artifact detection: they use first filtering at 11--140 hz, then Hilbert
# plotted raw data after the applied filter, and Hilbert - see cut artifacts in the beginning and end. (WJY arw we sure it  s not hilbert?)
# then, tried to only filter - very noisy data. but most likely the filtering is the source. Because of the cut-off in the beginning and end.
# Solutions: zero padding in the beginning and end before filtering, which will be cut off after. But may still create a jump while filtering and keep the artifact.
# Better: add 2s of dummy data at the beginning and end of the recording, and then crop it out (the data added should be mirrored). This will not create a jump in the filtering.
# Tried

# Problem found! 2 problems: 
# 1st: The main artifact is actually introduced by filtering power lines. filtering the data at 150 Hz (harmonics) clearly creates this artifact.
# Removed that and any other filtering over the range of muscle freqs, since we don't need them anyways. (over 140 Hz)
# 2nd: Still some artifact is present in the beginning and end of the recording. For this attach mirrored data on both ends.
# Then detect muscle, then cut the resulting scores away for the attached period.
# There will be still some very minimal artifact at the beginning/end of this attachment - probably due to the attachment itself: the mirrored data is still not a normally shaped signal.
# See example in cell above of how all 3 option look: oroginal, with attached data and with attached and cut away.

import numpy as np
import plotly.graph_objects as go
import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

#%matplotlib qt
raw_first = raw.copy().crop(tmin=0, tmax=1)
#raw_first.plot()


std_val=raw_first.get_data()

window = np.hanning(std_val.shape[-1])*std_val

window

freqs = np.fft.rfftfreq(window.shape[-1], 1/raw.info['sfreq'])

components = np.fft.fft(window, axis=-1)

components.shape

fig = go.Figure()
for ch in range(15, 250):
    fig.add_trace(go.Scatter(x=freqs, y = np.abs(components[ch, 0:500])))
fig.add_trace(go.Scatter(x=freqs, y = np.abs(components[0, 0:500])))
#from mne annotate_muscle_zscore:
from scipy.stats import zscore
from scipy.ndimage import label

filter_freq=(110, 140)
legend_category = 'mag'

raw_copy = raw_first.copy()
raw_copy.load_data()

if legend_category is None:
    raw_ch_type = raw_copy.get_channel_types()
    if 'mag' in raw_ch_type:
        legend_category = 'mag'
    elif 'grad' in raw_ch_type:
        legend_category = 'grad'
    elif 'eeg' in raw_ch_type:
        legend_category = 'eeg'
    else:
        raise ValueError('No M/EEG channel types found, please specify a'
                            ' ch_type or provide M/EEG sensor data')

if legend_category in ('mag', 'grad'):
    raw_copy.pick_types(meg=legend_category, ref_meg=False)
else:
    legend_category = {'meg': False, legend_category: True}
    raw_copy.pick_types(**legend_category)

#raw_copy.filter(filter_freq[0], filter_freq[1], fir_design='firwin',
#                pad="reflect_limited")

hilb_applied=raw_copy.apply_hilbert(envelope=True)
hilb_applied.plot()



In [ ]:
import numpy as np

# Load the list of values into a NumPy array
values = np.array([1, 2, 1.8, 2.5, 3, 3.5, 4, 3.8, 5, 4, 2, 2.1, 1, 0, 2, 4, 6])



import numpy as np
from scipy.signal import butter, filtfilt, savgol_filter, find_peaks

import plotly.graph_objects as go
import numpy as np

# Generate a noisy wave shape
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.5*np.random.randn(len(t))

data = np.random.randn(1000) #no wave shape
# Load the noisy wave data into a NumPy array
wave_data = data

# Create a Plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=wave_data, mode='lines', name='Noisy Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Noisy Wave Shape')
fig.show()

# Apply a low-pass filter to remove high-frequency noise
b, a = butter(5, 0.1, 'low')
filtered_data = filtfilt(b, a, wave_data)

# plot filtered data
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=filtered_data, mode='lines', name='Filtered Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Filtered Wave Shape')
fig.show()

# Apply a Savitzky-Golay filter to further reduce noise and extract the underlying wave shape
#smoothed_data = savgol_filter(wave_data, window_length=int(len(wave_data)/4), polyorder=3)
smoothed_data = savgol_filter(data, window_length=100, polyorder=3)

# Identify the shape of the wave using peak detection or curve fitting
# For example, you can use the `scipy.signal.find_peaks` function to detect peaks in the smoothed data
#peaks, _ = find_peaks(smoothed_data, height=0.5*np.max(smoothed_data))
peaks, _ = find_peaks(smoothed_data)

# plot smoothed data
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=smoothed_data, mode='lines', name='Smoothed Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Smoothed Wave Shape')
fig.add_trace(go.Scatter(x=t[peaks], y=smoothed_data[peaks], mode='markers', name='Peaks'))
fig.show()



In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

# Generate a noisy wave shape
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.5*np.random.randn(len(t))

# Find the peaks in the wave
peaks, _ = find_peaks(y)

# Count the number of peaks
num_peaks = len(peaks)

# Create a Plotly figure
fig = go.Figure()

# Add the noisy wave shape to the figure
fig.add_trace(go.Scatter(x=t, y=y, mode='lines', name='Noisy Wave'))

# Add the peaks to the figure
fig.add_trace(go.Scatter(x=t[peaks], y=y[peaks], mode='markers', name='Peaks'))

# Add axis labels and a title
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Noisy Wave Shape')

# Show the figure and print the number of peaks
fig.show()
print(f'The wave has {num_peaks} crest(s).')

In [ ]:
import mne
picks_EOG = mne.pick_types(raw.info, eog=True)
eog_ch_name = [raw.info['chs'][name]['ch_name'] for name in picks_EOG]
eog_ch_name

In [ ]:
one_psd = [2.93686870e-12, 5.37336497e-13, 2.34749324e-13, 1.70403629e-13
, 1.42868936e-13, 1.10614848e-13, 1.01586902e-13, 9.41699507e-14
, 8.41904711e-14, 7.56254639e-14, 6.98933286e-14, 6.47116338e-14
, 5.37107007e-14, 5.42174045e-14, 4.78692577e-14, 4.36476164e-14
, 3.69272073e-14, 3.81479068e-14, 4.07614720e-14, 3.71683505e-14
, 3.74843265e-14, 3.57210926e-14, 3.99173535e-14, 4.87143053e-14
, 4.20066645e-14, 3.84896719e-14, 3.13482998e-14, 2.86289627e-14
, 2.82586165e-14, 2.71780036e-14, 2.48692250e-14, 2.71251350e-14
, 2.79561808e-14, 2.72047767e-14, 2.79637330e-14, 2.55955578e-14
, 2.53291180e-14, 2.01500680e-14, 2.12080778e-14, 2.18529602e-14
, 2.12775368e-14, 2.14334140e-14, 2.18751322e-14, 1.97884378e-14
, 1.89952388e-14, 1.79404586e-14, 2.01555022e-14, 2.21105500e-14
, 1.87897255e-14, 1.95585625e-14, 2.07067862e-14, 2.15786185e-14
, 1.78539522e-14, 1.89461927e-14, 1.83714513e-14, 1.91272285e-14
, 1.81790918e-14, 1.51935998e-14, 1.55331746e-14, 1.37627320e-14
, 1.37333973e-14, 1.47261781e-14, 1.35323358e-14, 1.23234074e-14
, 1.26296023e-14, 1.39794705e-14, 1.32391885e-14, 1.33172509e-14
, 1.40752537e-14, 1.35291881e-14, 1.46771014e-14, 1.57039580e-14
, 1.76870590e-14, 2.11409680e-14, 2.66470647e-14, 2.09066429e-14
, 1.68226688e-14, 1.63034232e-14, 1.32317697e-14, 1.20372472e-14
, 1.10395275e-14, 1.17336558e-14, 1.12817157e-14, 1.31068881e-14
, 1.36940739e-14, 1.48016686e-14, 1.35999052e-14, 1.56644411e-14
, 1.51726149e-14, 1.95274934e-14, 1.84669709e-14, 1.89443054e-14
, 1.82544652e-14, 1.92617658e-14, 1.80902967e-14, 2.17239287e-14
, 2.67917043e-14, 4.45194367e-14, 2.01857457e-12, 4.40594585e-12
, 1.74602155e-12, 4.30562941e-14, 2.53461498e-14, 1.87278757e-14
, 1.54232144e-14, 1.67924147e-14, 1.22749773e-14, 1.25017017e-14
, 1.22475994e-14, 1.02921463e-14, 1.07700101e-14, 1.02658035e-14
, 9.54949869e-15, 9.84280694e-15, 8.88745310e-15, 9.02206922e-15
, 8.47210049e-15, 8.64491709e-15, 1.32254861e-14, 1.89573615e-14
, 1.23079196e-14, 8.62994931e-15, 8.12535185e-15, 8.01035318e-15
, 7.53220890e-15, 8.02056256e-15, 7.90231409e-15, 7.63270083e-15
, 7.93212379e-15, 7.28368608e-15, 7.59772607e-15, 7.26136429e-15
, 7.86504197e-15, 7.36256360e-15, 7.02847343e-15, 7.08620266e-15
, 6.86068169e-15, 7.21792433e-15, 7.28674098e-15, 6.88181371e-15
, 6.78751472e-15, 6.59002904e-15, 6.74850515e-15, 6.53743454e-15
, 6.70834535e-15, 6.72520433e-15, 6.78371437e-15, 6.70420118e-15
, 6.97442862e-15, 7.26767528e-15, 6.79388360e-15, 6.83101277e-15
, 6.90684197e-15, 6.45716620e-15, 6.66190889e-15, 6.49304182e-15
, 6.38068712e-15, 6.29160702e-15, 5.92354089e-15, 6.33890242e-15
, 6.33787606e-15, 5.76688121e-15, 6.31821916e-15, 6.34536916e-15
, 6.51250512e-15, 6.43164190e-15, 6.46530769e-15, 6.44724883e-15
, 7.48305304e-15, 7.50925230e-15, 6.28419317e-15, 6.33908319e-15
, 5.86954984e-15, 6.54561206e-15, 6.08872456e-15, 6.40874736e-15
, 5.95870142e-15, 6.13488554e-15, 5.83721527e-15, 5.87793931e-15
, 5.81207088e-15, 5.98748087e-15, 5.94551525e-15, 5.75415575e-15
, 5.66968278e-15, 6.11006036e-15, 5.72066372e-15, 5.96629716e-15
, 5.80372053e-15, 5.75583336e-15, 5.84628922e-15, 5.63642362e-15
, 5.34942930e-15, 5.75920960e-15, 6.05337029e-15, 6.61372576e-15
, 7.14210116e-15, 6.94968538e-15, 1.85742697e-14, 3.52090532e-14
, 1.48785528e-14, 6.23577963e-15, 5.66229647e-15, 5.39212323e-15
, 5.32890121e-15, 5.54967559e-15, 5.29485491e-15, 5.65665900e-15
, 5.31337965e-15, 5.30139224e-15, 5.21434237e-15, 5.61739646e-15
, 5.62673191e-15, 5.68441483e-15, 5.43332729e-15, 5.34563989e-15
, 5.69510011e-15, 6.43038706e-15, 5.52069097e-15, 5.22891308e-15
, 5.06513758e-15, 5.15715319e-15, 5.32484298e-15, 5.37071225e-15
, 5.24099974e-15, 5.14413780e-15, 5.05322799e-15, 5.27277366e-15
, 5.17209094e-15, 5.19895605e-15, 5.04662049e-15, 5.13492402e-15
, 5.39573281e-15, 5.13639899e-15, 5.29696474e-15, 5.29749076e-15
, 5.23187737e-15, 5.14179424e-15, 1.44011463e-14, 2.29753019e-14
, 8.85041560e-15, 5.16398069e-15, 5.09075672e-15, 5.06681957e-15
, 5.17284653e-15, 4.99688083e-15, 5.01988585e-15, 5.07952302e-15
, 4.97188381e-15, 5.17733558e-15, 4.97124292e-15, 4.96910679e-15
, 4.96283207e-15, 5.07193856e-15, 4.80712108e-15, 4.97630935e-15
, 4.93727883e-15, 4.84091247e-15, 5.07370238e-15, 4.76459850e-15
, 4.86678392e-15, 5.03907955e-15, 4.91645908e-15, 4.99691785e-15
, 4.81326372e-15, 5.56398292e-15, 5.40286001e-15, 4.91762834e-15
, 4.96042200e-15, 4.86125369e-15, 5.04306529e-15, 4.89229744e-15
, 4.93924928e-15, 4.91889752e-15, 4.92336366e-15, 4.91476256e-15
, 4.90731383e-15, 4.79751988e-15, 4.96181659e-15, 5.04353794e-15]

#plot the data with ploty:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

freqs = [i/2 for i in range(0, 280)]
prominence_lvl_pos = 50
prominence_pos=(max(one_psd) - min(one_psd)) / prominence_lvl_pos
noisy_freqs_indexes, _ = find_peaks(one_psd, prominence=prominence_pos)
noisy_freqs_indexes = [int(i) for i in noisy_freqs_indexes]

for i in range(0,2):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=freqs, y=one_psd, name='psd'))
    fig.update_layout(title=' PSD', xaxis_title='Frequency', yaxis_title='Amplitude',
            yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e'))

    fig.add_trace(go.Scatter(x=[freqs[noisy_freqs_indexes[0]]], y=[one_psd[noisy_freqs_indexes[0]]], mode='markers', name='peaks'))

    if i == 0:
        fig.update_yaxes(type="log")

    fig.show()


In [ ]:
df_head_pos


In [ ]:
bool('False')

#convert "false" to boolean:
import ast
t = ast.literal_eval('False')
t
